# Nornir Tutorial

## Register plugins


In [2]:
from nornir.core.plugins.inventory import InventoryPluginRegister
from nornir.plugins.runners import ThreadedRunner

InventoryPluginRegister.register("ThreadedRunner", ThreadedRunner)

## Initializing Nornir

In [3]:
from nornir import InitNornir
config_file = !pwd
nr = InitNornir(config_file = config_file[0] + '/config.yaml')

In [4]:
nr.config.runner.options

{'num_workers': 100}

## Inventory 

In [5]:
from nornir.core.inventory import Host
import json
print(json.dumps(Host.schema(), indent=4))


{
    "name": "str",
    "connection_options": {
        "$connection_type": {
            "extras": {
                "$key": "$value"
            },
            "hostname": "str",
            "port": "int",
            "username": "str",
            "password": "str",
            "platform": "str"
        }
    },
    "groups": [
        "$group_name"
    ],
    "data": {
        "$key": "$value"
    },
    "hostname": "str",
    "port": "int",
    "username": "str",
    "password": "str",
    "platform": "str"
}


In [6]:
nr.inventory.hosts

{'R1': Host: R1,
 'R2': Host: R2,
 'R3': Host: R3,
 'R4': Host: R4,
 'R5': Host: R5,
 'R6': Host: R6,
 'OOB-switch': Host: OOB-switch,
 'NMS': Host: NMS}

In [7]:
nr.inventory.groups

{'SiteA': Group: SiteA, 'SiteB': Group: SiteB, 'OOB': Group: OOB}

In [8]:
R1 = nr.inventory.hosts['R1']
print(R1.items())
R1['site']

dict_items([('site', 'SiteA'), ('role', 'BRANCH'), ('type', 'network_device'), ('domain', 'Cola'), ('asn', 65001)])


'SiteA'

In [9]:
nr.inventory.groups

{'SiteA': Group: SiteA, 'SiteB': Group: SiteB, 'OOB': Group: OOB}

## Tasks

In [10]:
nr = nr.filter(site = 'SiteA')
from nornir.core.task import Task, Result
from nornir_utils.plugins.functions import print_result

def hello_world(task: Task) -> Result:
    return Result(
        host=task.host,
        result=f"{task.host.name} says hello world!"
    )
result = nr.run(task=hello_world)
print_result(result)


hello_world*********************************************************************
* R1 ** changed : False ********************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
R1 says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* R2 ** changed : False ********************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
R2 says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* R3 ** changed : False ********************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
R3 says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [11]:

def say(task: Task, text: str) -> Result:
    return Result(
        host=task.host,
        result=f"{task.host.name} says {text}"
    )

result = nr.run(
    name="Saying goodbye in a very friendly manner",
    task=say,
    text="buhbye!"
)
print_result(result)

Saying goodbye in a very friendly manner****************************************
* R1 ** changed : False ********************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
R1 says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* R2 ** changed : False ********************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
R2 says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* R3 ** changed : False ********************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
R3 says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Grouping tasks

In [12]:
def count(task: Task, number: int) -> Result:
    return Result(
        host=task.host,
        result=f"{[n for n in range(0, number)]}"
    )

def greet_and_count(task: Task, number: int) -> Result:
    task.run(
        name="Greeting is the polite thing to do",
        task=say,
        text="hi!",
    )

    task.run(
        name="Counting beans",
        task=count,
        number=number,
    )
    task.run(
        name="We should say bye too",
        task=say,
        text="bye!",
    )

    # let's inform if we counted even or odd times
    even_or_odds = "even" if number % 2 == 1 else "odd"
    return Result(
        host=task.host,
        result=f"{task.host} counted {even_or_odds} times!",
    )

result = nr.run(
    name="Counting to 5 while being very polite",
    task=greet_and_count,
    number=5,
)
print_result(result)

Counting to 5 while being very polite*******************************************
* R1 ** changed : False ********************************************************
vvvv Counting to 5 while being very polite ** changed : False vvvvvvvvvvvvvvvvvv INFO
R1 counted even times!
---- Greeting is the polite thing to do ** changed : False --------------------- INFO
R1 says hi!
---- Counting beans ** changed : False ----------------------------------------- INFO
[0, 1, 2, 3, 4]
---- We should say bye too ** changed : False ---------------------------------- INFO
R1 says bye!
^^^^ END Counting to 5 while being very polite ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* R2 ** changed : False ********************************************************
vvvv Counting to 5 while being very polite ** changed : False vvvvvvvvvvvvvvvvvv INFO
R2 counted even times!
---- Greeting is the polite thing to do ** changed : False --------------------- INFO
R2 says hi!
---- Counting beans ** changed : False ----------------------

## Processing results

In [27]:
import logging

from nornir import InitNornir
from nornir.core.task import Task, Result

# instantiate the nr object
nr = InitNornir(config_file="config.yaml")
# let's filter it down to simplify the output
cmh = nr.filter(site="SiteA")

def count(task: Task, number: int) -> Result:
    return Result(
        host=task.host,
        result=f"{[n for n in range(0, number)]}"
    )

def say(task: Task, text: str) -> Result:
    if task.host.name == "R1":
        raise Exception("I can't say anything right now")
    return Result(
        host=task.host,
        result=f"{task.host.name} says {text}"
    )

In [28]:
def greet_and_count(task: Task, number: int):
    task.run(
        name="Greeting is the polite thing to do",
        severity_level=logging.DEBUG,
        task=say,
        text="hi!",
    )

    task.run(
        name="Counting beans",
        task=count,
        number=number,
    )
    task.run(
        name="We should say bye too",
        severity_level=logging.DEBUG,
        task=say,
        text="bye!",
    )

    # let's inform if we counted even or odd times
    even_or_odds = "even" if number % 2 == 1 else "odd"
    return Result(
        host=task.host,
        result=f"{task.host} counted {even_or_odds} times!",
    )

In [29]:
result = cmh.run(
    task=greet_and_count,
    number=6,
)

### The easy way


In [30]:
from nornir_utils.plugins.functions import print_result

print_result(result)

greet_and_count*****************************************************************
* R1 ** changed : False ********************************************************
vvvv greet_and_count ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
Subtask: Greeting is the polite thing to do (failed)

---- Greeting is the polite thing to do ** changed : False --------------------- ERROR
Traceback (most recent call last):
  File "/home/kali/Documents/PYTHON_BASIC/venv/lib/python3.9/site-packages/nornir/core/task.py", line 98, in start
    r = self.task(self, **self.params)
  File "<ipython-input-27-92b919bcb2f8>", line 19, in say
    raise Exception("I can't say anything right now")
Exception: I can't say anything right now

^^^^ END greet_and_count ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* R2 ** changed : False ********************************************************
vvvv greet_and_count ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
R2 counted odd tim

print a single host

In [32]:
print_result(result["R1"])

vvvv R1: greet_and_count ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
Subtask: Greeting is the polite thing to do (failed)

---- Greeting is the polite thing to do ** changed : False --------------------- ERROR
Traceback (most recent call last):
  File "/home/kali/Documents/PYTHON_BASIC/venv/lib/python3.9/site-packages/nornir/core/task.py", line 98, in start
    r = self.task(self, **self.params)
  File "<ipython-input-27-92b919bcb2f8>", line 19, in say
    raise Exception("I can't say anything right now")
Exception: I can't say anything right now

^^^^ END greet_and_count ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


print a single task

serverity_level:
- DEBUG : 10
- INFO : 20
- WARNING : 30
- ERROR : 40
- FATAL : 50

In [33]:
print_result(result["R2"][2])

---- R2: Counting beans ** changed : False ------------------------------------- INFO
[0, 1, 2, 3, 4, 5]


In [50]:
print_result(result, severity_level=logging.INFO)

greet_and_count*****************************************************************
* R1 ** changed : False ********************************************************
vvvv greet_and_count ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
Subtask: Greeting is the polite thing to do (failed)

---- Greeting is the polite thing to do ** changed : False --------------------- ERROR
Traceback (most recent call last):
  File "/home/kali/Documents/PYTHON_BASIC/venv/lib/python3.9/site-packages/nornir/core/task.py", line 98, in start
    r = self.task(self, **self.params)
  File "<ipython-input-27-92b919bcb2f8>", line 19, in say
    raise Exception("I can't say anything right now")
Exception: I can't say anything right now

^^^^ END greet_and_count ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* R2 ** changed : False ********************************************************
vvvv greet_and_count ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
R2 counted odd tim